In [1]:
# 一个典型的PB文件保存示例

In [2]:
import os
import tensorflow as tf
from tensorflow.python.framework import graph_util

/Users/zoe/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
pb_file_path = os.getcwd()

In [10]:
with tf.Session(graph = tf.Graph()) as sess:
    x = tf.placeholder(tf.int32, name='x')
    y = tf.placeholder(tf.int32, name='y')
    b = tf.Variable(1, name='b')
    xy = tf.multiply(x,y)
    
    op = tf.add(xy,b,name='op_to_store') # 这里的op需要加上name属性
    
    sess.run(tf.global_variables_initializer())
    
    # convert_variables_to_constants
    # 需要指定output_node_names，list()，可以多个
    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph_def, ['op_to_store'])
    
    # 测试OP
    feed_dict = {x:10,y:3}
    print('>>>> TEST:',sess.run(op,feed_dict))
    
    # 写入序列化的PB文件
    with tf.gfile.FastGFile(pb_file_path+'model.pb',mode='wb') as f:
        f.write(constant_graph.SerializeToString())

INFO:tensorflow:Froze 1 variables.
Converted 1 variables to const ops.
>>>> TEST: 31


In [ ]:
# -------保存成功---------

In [ ]:
# 加载PB模型文件代码

In [12]:
from tensorflow.python.platform import gfile

In [13]:
sess = tf.Session()

In [14]:
with gfile.FastGFile(pb_file_path+'model.pb','rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='') # 导入计算图

In [15]:
# 初始化
sess.run(tf.global_variables_initializer())

In [16]:
# 先复原变量
print(sess.run('b:0'))

1


In [17]:
# 输入
input_x = sess.graph.get_tensor_by_name('x:0')
input_y = sess.graph.get_tensor_by_name('y:0')

In [18]:
op = sess.graph.get_tensor_by_name('op_to_store:0')

In [19]:
ret = sess.run(op, feed_dict={input_x:5,input_y:5})

In [20]:
print(ret)

26


In [21]:
# ===========以上相当于加载了PB文件的计算图结构==========

In [ ]:
# PB文件保存方式二

In [23]:
# 另外保存为save model格式也可以生成PB文件，并且更加简单

In [36]:
import os
import tensorflow as tf
from tensorflow.python.framework import graph_util
pb_file_path = os.getcwd()
with tf.Session(graph=tf.Graph()) as sess:
    x = tf.placeholder(tf.int32,name='x')
    y = tf.placeholder(tf.int32,name='y')
    b = tf.Variable(1,name='b')
    xy = tf.multiply(x,y)
    op = tf.add(xy,b,name='op_to_store')
    
    sess.run(tf.global_variables_initializer())
    
    constant_graph = graph_util.convert_variables_to_constants(sess,sess.graph_def,['op_to_store'])
    
    feed_dict = {x:10,y:3}
    print('>>>> TEST:',sess.run(op,feed_dict))
    
#     with tf.gfile.FastGFile(pb_file_path+'model.pb',mode='wb') as f:
#         f.write(constant_graph.SerializeToString())
        
    builder = tf.saved_model.builder.SavedModelBuilder(pb_file_path+'savemodel')
    # 构造模型保存的内容，指定要保存的session,特定的tag,输入输出信息字典，额外的信息
    builder.add_meta_graph_and_variables(sess,['cpu_server_1'])
builder.save()

INFO:tensorflow:Froze 1 variables.
Converted 1 variables to const ops.
>>>> TEST: 31
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'/Users/zoe/PycharmProjects/Zoe_NLP/dl_TensorFlowsavemodel/saved_model.pb'


b'/Users/zoe/PycharmProjects/Zoe_NLP/dl_TensorFlowsavemodel/saved_model.pb'

In [37]:
# =====保存成功=====>>>> 生成savemodel文件夹，生成saved_model.pb文件和variables文件夹
# saved_model.pb用于保存模型结构等信息
# variables保存所有变量。

In [38]:
# 对应的模型导入方法

In [39]:
with tf.Session(graph=tf.Graph()) as sess:
    # 只需要指定加载模型的session,模型的tag,模型的保存路径即可
    tf.saved_model.loader.load(sess,['cpu_server_1'],pb_file_path+'savemodel')
    sess.run(tf.global_variables_initializer())
    
    input_x = sess.graph.get_tensor_by_name('x:0')
    input_y = sess.graph.get_tensor_by_name('y:0')
    
    op = sess.graph.get_tensor_by_name('op_to_store:0')
    
    ret = sess.run(op, feed_dict={input_x:10, input_y:5})
    print('>>>>',ret)

INFO:tensorflow:Restoring parameters from b'/Users/zoe/PycharmProjects/Zoe_NLP/dl_TensorFlowsavemodel/variables/variables'
>>>> 51


In [40]:
# 以上两种模型的加载方式中都要知道tensor的name。
# 那么如何可以在不知道tensor name的情况下使用呢，实现彻底的解耦呢？
# 在保存时给add_meta_graph_and_variables方法传入第三个参数，signature_def_map即可。

In [41]:
# ============拓展=========

In [42]:
# 保存为ckpt的时候，直接加载网络结构的使用方法

In [43]:
def restore_model_ckpt(ckpt_file_path):
    '''保存ckpt格式的文件'''
    sess = tf.Session()
    
    # 加载模型结构
    saver = tf.train.import_meta_graph('./ckpt/model.ckpt.meta')
    # 只需要指定目录就可以恢复所有变量信息
    saver.restore(sess,tf.train.latest_checkpoint('.ckpt'))
    
    # 直接获取保存的变量
    print(sess.run('b:0'))
    
    # 获取placeholder变量
    input_x = sess.graph.get_tensor_by_name('x:0')
    input_y = sess.graph.get_tensor_by_name('y:0')
    
    # 获取需要进行计算的operator
    op = sess.graph.get_tensor_by_name('op_to_store:0')
    
    # 加入新的操作
    add_on_op = tf.multiply(op, 2)
    
    ret = sess.run(add_on_op, {input_x: 5, input_y: 5})
    print(ret) 